In [14]:
import pandas as pd
import numpy as np
# Load dataset
df = pd.read_csv('dataset.csv')

# Show shape and sample
print("Shape of dataset:", df.shape)
df.head()


Shape of dataset: (18368, 12)


,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00


In [15]:
print(df.columns.tolist())
df.info()


['ID', 'SystemCodeNumber', 'Capacity', 'Latitude', 'Longitude', 'Occupancy', 'VehicleType', 'TrafficConditionNearby', 'QueueLength', 'IsSpecialDay', 'LastUpdatedDate', 'LastUpdatedTime']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18368 entries, 0 to 18367
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID                      18368 non-null  int64  
 1   SystemCodeNumber        18368 non-null  object 
 2   Capacity                18368 non-null  int64  
 3   Latitude                18368 non-null  float64
 4   Longitude               18368 non-null  float64
 5   Occupancy               18368 non-null  int64  
 6   VehicleType             18368 non-null  object 
 7   TrafficConditionNearby  18368 non-null  object 
 8   QueueLength             18368 non-null  int64  
 9   IsSpecialDay            18368 non-null  int64  
 10  LastUpdatedDate         18368 non-null  object 
 11  LastUpdate

In [16]:
# import pandas as pd

# # Load the dataset
# df = pd.read_csv("/mnt/data/dataset.csv")

# --- Data Preprocessing ---
# Merge date and time into a single datetime column
df['timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], dayfirst=True)

# Compute occupancy ratio
df['occupancy_ratio'] = df['Occupancy'] / df['Capacity']

# Map traffic condition to numeric levels
traffic_map = {'low': 1, 'medium': 2, 'high': 3}
df['traffic_level'] = df['TrafficConditionNearby'].map(traffic_map)

# Map vehicle type to weights
vehicle_map = {'bike': 0.5, 'car': 1, 'truck': 1.5}
df['vehicle_weight'] = df['VehicleType'].map(vehicle_map)

# --- Model 1: Baseline Linear Pricing ---
BASE_PRICE = 10
ALPHA = 5

# Sort data by lot and timestamp
df_sorted = df.sort_values(by=['SystemCodeNumber', 'timestamp']).copy()

# Initialize Model1_Price column with base price
df_sorted['Model1_Price'] = BASE_PRICE

# Function to apply Model 1 pricing per group
def apply_model1(group):
    prices = [BASE_PRICE]
    for i in range(1, len(group)):
        prev_price = prices[-1]
        occ_ratio = group.iloc[i]['occupancy_ratio']
        new_price = prev_price + ALPHA * occ_ratio
        prices.append(new_price)
    group['Model1_Price'] = prices
    return group

# Apply pricing model group-wise
df_model1 = df_sorted.groupby('SystemCodeNumber', group_keys=False).apply(apply_model1)

# Preview the result
print(df_model1[['SystemCodeNumber', 'timestamp', 'occupancy_ratio', 'Model1_Price']].head(10))


  SystemCodeNumber           timestamp  occupancy_ratio  Model1_Price
0      BHMBCCMKT01 2016-10-04 07:59:00         0.105719     10.000000
1      BHMBCCMKT01 2016-10-04 08:25:00         0.110919     10.554593
2      BHMBCCMKT01 2016-10-04 08:59:00         0.138648     11.247834
3      BHMBCCMKT01 2016-10-04 09:32:00         0.185442     12.175043
4      BHMBCCMKT01 2016-10-04 09:59:00         0.259965     13.474870
5      BHMBCCMKT01 2016-10-04 10:26:00         0.306759     15.008666
6      BHMBCCMKT01 2016-10-04 10:59:00         0.379549     16.906412
7      BHMBCCMKT01 2016-10-04 11:25:00         0.428076     19.046794
8      BHMBCCMKT01 2016-10-04 11:59:00         0.448873     21.291161
9      BHMBCCMKT01 2016-10-04 12:29:00         0.461005     23.596187


C:\Users\girijesh\AppData\Local\Temp\ipykernel_12236\1942229406.py:43: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_model1 = df_sorted.groupby('SystemCodeNumber', group_keys=False).apply(apply_model1)


In [17]:


# Define coefficients
ALPHA = 0.5
BETA = 0.3
GAMMA = 0.2
DELTA = 0.1
EPSILON = 0.1
LAMBDA = 0.8
BASE_PRICE = 10

# Compute demand function
df_model1['raw_demand'] = (
    ALPHA * df_model1['occupancy_ratio'] +
    BETA * df_model1['QueueLength'] -
    GAMMA * df_model1['traffic_level'] +
    DELTA * df_model1['IsSpecialDay'] +
    EPSILON * df_model1['vehicle_weight']
)

# Normalize demand (min-max normalization within each lot)
def normalize_demand(group):
    min_d = group['raw_demand'].min()
    max_d = group['raw_demand'].max()
    group['normalized_demand'] = (group['raw_demand'] - min_d) / (max_d - min_d + 1e-6)
    return group

df_model2 = df_model1.groupby('SystemCodeNumber', group_keys=False).apply(normalize_demand)

# Compute final Model 2 price (bounded between 0.5x and 2x base price)
df_model2['Model2_Price'] = BASE_PRICE * (1 + LAMBDA * df_model2['normalized_demand'])
df_model2['Model2_Price'] = df_model2['Model2_Price'].clip(lower=BASE_PRICE * 0.5, upper=BASE_PRICE * 2.0)

# Preview results
df_model2[['SystemCodeNumber', 'timestamp', 'occupancy_ratio', 'QueueLength', 'traffic_level',
           'vehicle_weight', 'raw_demand', 'normalized_demand', 'Model2_Price']].head(10)


C:\Users\girijesh\AppData\Local\Temp\ipykernel_12236\2073974370.py:26: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_model2 = df_model1.groupby('SystemCodeNumber', group_keys=False).apply(normalize_demand)


,SystemCodeNumber,timestamp,occupancy_ratio,QueueLength,traffic_level,vehicle_weight,raw_demand,normalized_demand,Model2_Price
0,BHMBCCMKT01,2016-10-04 07:59:00,0.105719,1,1.0,1.0,0.252860,0.120385,10.963081
1,BHMBCCMKT01,2016-10-04 08:25:00,0.110919,1,1.0,1.0,0.255459,0.121181,10.969446
2,BHMBCCMKT01,2016-10-04 08:59:00,0.138648,2,1.0,1.0,0.569324,0.217245,11.737958
3,BHMBCCMKT01,2016-10-04 09:32:00,0.185442,2,1.0,1.0,0.592721,0.224406,11.795247
4,BHMBCCMKT01,2016-10-04 09:59:00,0.259965,2,1.0,0.5,0.579983,0.220507,11.764056
5,BHMBCCMKT01,2016-10-04 10:26:00,0.306759,3,1.0,1.0,0.953380,0.334792,12.678336
6,BHMBCCMKT01,2016-10-04 10:59:00,0.379549,6,3.0,1.5,1.539775,0.514269,14.114151
7,BHMBCCMKT01,2016-10-04 11:25:00,0.428076,5,NaN,1.0,NaN,NaN,NaN
8,BHMBCCMKT01,2016-10-04 11:59:00,0.448873,5,NaN,NaN,NaN,NaN,NaN
9,BHMBCCMKT01,2016-10-04 12:29:00,0.461005,8,3.0,0.5,2.080503,0.679769,15.438148


In [18]:


def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  # Earth radius in meters
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)

    a = np.sin(delta_phi / 2.0)**2 + \
        np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2.0)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    return R * c  # in meters


In [19]:
# Get unique lot locations
lot_locations = df_model2[['SystemCodeNumber', 'Latitude', 'Longitude']].drop_duplicates().reset_index(drop=True)

# Dictionary to store nearby competitors for each lot
nearby_lots = {}

for i, row_i in lot_locations.iterrows():
    lot_i = row_i['SystemCodeNumber']
    lat_i, lon_i = row_i['Latitude'], row_i['Longitude']
    nearby = []
    for j, row_j in lot_locations.iterrows():
        if i == j:
            continue
        dist = haversine(lat_i, lon_i, row_j['Latitude'], row_j['Longitude'])
        if dist <= 500:
            nearby.append(row_j['SystemCodeNumber'])
    nearby_lots[lot_i] = nearby


In [20]:
# Initialize new column
df_model2['Model3_Price'] = df_model2['Model2_Price']
df_model2['RerouteSuggestion'] = False

# Function to adjust price based on competition
def adjust_price_with_competitors(row, df_grouped):
    lot = row['SystemCodeNumber']
    timestamp = row['timestamp']
    nearby = nearby_lots.get(lot, [])
    
    # Get competitor prices at the same timestamp
    competitor_prices = df_grouped.loc[(df_grouped['SystemCodeNumber'].isin(nearby)) &
                                       (df_grouped['timestamp'] == timestamp), 'Model2_Price']
    
    if len(competitor_prices) == 0:
        return row['Model2_Price'], False

    avg_comp_price = competitor_prices.mean()

    # Price adjustment
    adjusted_price = row['Model2_Price']
    reroute = False

    if row['occupancy_ratio'] > 0.95 and avg_comp_price < row['Model2_Price']:
        adjusted_price *= 0.9  # reduce price to attract customers
        reroute = True         # suggest rerouting
    elif avg_comp_price > row['Model2_Price']:
        adjusted_price *= 1.1  # increase price slightly

    # Clamp final price
    adjusted_price = max(BASE_PRICE * 0.5, min(BASE_PRICE * 2.0, adjusted_price))
    return adjusted_price, reroute

# Apply row-wise
df_model2[['Model3_Price', 'RerouteSuggestion']] = df_model2.apply(
    lambda row: adjust_price_with_competitors(row, df_model2), axis=1, result_type='expand'
)


In [21]:
df_model2.head

<bound method NDFrame.head of           ID SystemCodeNumber  Capacity   Latitude  Longitude  Occupancy  \
0          0      BHMBCCMKT01       577  26.144536  91.736172         61   
1          1      BHMBCCMKT01       577  26.144536  91.736172         64   
2          2      BHMBCCMKT01       577  26.144536  91.736172         80   
3          3      BHMBCCMKT01       577  26.144536  91.736172        107   
4          4      BHMBCCMKT01       577  26.144536  91.736172        150   
...      ...              ...       ...        ...        ...        ...   
18363  18363         Shopping      1920  26.150504  91.733531       1517   
18364  18364         Shopping      1920  26.150504  91.733531       1487   
18365  18365         Shopping      1920  26.150504  91.733531       1432   
18366  18366         Shopping      1920  26.150504  91.733531       1321   
18367  18367         Shopping      1920  26.150504  91.733531       1180   

      VehicleType TrafficConditionNearby  QueueLength  Is

In [22]:
import pathway as pw

import datetime

# Load data into a pandas DataFrame first (reuse df_model3 from before)
input_data = df_model2[['SystemCodeNumber', 'timestamp', 'Model3_Price']]

# Create a simulated stream from pandas
table = pw.io.pandas.read_pandas(
    input_data,
    schema=pw.schema(
        SystemCodeNumber=pw.str,
        timestamp=pw.timestamp,
        Model3_Price=pw.float64,
    ),
    autocommit_duration_ms=500,  # simulate streaming
    mode='streaming_with_updates',
    primary_key='timestamp'
)


This is not the real Pathway package.
Visit https://pathway.com/developers/ to get Pathway.
Already tried that? Visit https://pathway.com/troubleshooting/ to get help.
Note: your platform is Windows-10-10.0.26100-SP0, your Python is CPython 3.11.0.


AttributeError: module 'pathway' has no attribute 'io'
This is not the real Pathway package.
Visit https://pathway.com/developers/ to get Pathway.
Already tried that? Visit https://pathway.com/troubleshooting/ to get help.
Note: your platform is Windows-10-10.0.26100-SP0, your Python is CPython 3.11.0.

In [ ]:
class PriceStream(pw.Class):
    SystemCodeNumber: pw.str
    timestamp: pw.timestamp
    Model3_Price: pw.float64

# Map Pathway table to your class
prices = table.select(
    SystemCodeNumber=table.SystemCodeNumber,
    timestamp=table.timestamp,
    Model3_Price=table.Model3_Price
).with_class(PriceStream)


In [ ]:
# Print to console
pw.io.jsonlines.write_jsonlines(prices, dest='stream_output.jsonl')
